# Parameter Sweeps - Second Attempt

In [1]:
import kelp_param
import kelp3d_objs as k3
import ipyparallel as ipp

In [2]:
remote_config = kelp_param.ParamSpanRemoteConfig()
c = ipp.Client()
lv = c.load_balanced_view()
dv = c.direct_view()

In [3]:
def cdkelp():
    import os
    os.chdir('/global/homes/o/oevans/kelp/code/python')
dv.apply(cdkelp)

<AsyncResult: cdkelp>

In [ ]:
print(f.sent)
print(f.started)
print(f.done())
c.queue_status()

In [ ]:
f.cancel()

In [4]:
# Hard-coded parameters
absorptance_kelp = 0.8 # (%)
rope_spacing = 5.0 # (m)
zmin, zmax = 0.0, 10.0 # (m)
nz=10

# Light from above
I0 = 50.0 # (W/m^2)
phi_s = 0 # (rad)
theta_s = 0 # (rad)
decay = 0 # (?) - light (practically) only from directly above

# Calculated or otherwise boring parameters
xmin = -rope_spacing/2
xmax = rope_spacing/2
ymin = -rope_spacing/2
ymax = rope_spacing/2

# Solver parameters
tol_abs = 1e-6
tol_rel = 1e-6
maxiter_inner = 50
maxiter_outer = 50
gmres_wrapper = k3.gmres_wrapper

const = (rope_spacing, zmin, zmax, nz, I0, phi_s, theta_s, decay, xmin, xmax, ymin, ymax, absorptance_kelp,
         tol_abs, tol_rel, maxiter_inner, maxiter_outer, gmres_wrapper)

In [7]:
psw = kelp_param.ParamSpanWidget(
    'kelp', 
    remote_config, 
    kelp_param.kelp_calculate, 
    kelp_param.kelp_visualize
)
psw.set_params(
    a_water = [0.5],
    b = [0.5, 0.1],#, 0.7],
    # Number of x & y grid cells
    ns = [24],#, 10, 12],
    # Number of angular grid cells (Same for theta & phi)
    na = [24],#, 10, 12],#, 14, 18, 20, 24],
    #na = [10, 12, 14, 18],
    gmres_flag = [True],
    num_scatters = [0],
    kelp_profile = ['top-heavy'],# 'bottom-heavy', 'uniform'],
    const=[const]
)
psw.submit_computations()
psw

ParamSpanWidget(children=(Output(layout=Layout(border='1px solid', height='500px', overflow_x='scroll', overfl…

In [ ]:
[f.started for f in psw.compute_futures]

In [ ]:
c.queue_status()

In [ ]:
%px print("hi")

In [ ]:
def block_mean(large_arr, small_shape):
    """Calculate an array of block means of `large_arr` which has the shape `small_shape`"""

    if all(n_large % n_small == 0 for n_small, n_large in zip(small_shape, large_arr.shape)):
        # Try to abstract over number of dimensions
        avg = np.zeros(small_shape)
        block_sizes = [n_large // n_small for n_small, n_large in zip(small_shape, large_arr.shape)]
        #print("block_sizes = {}".format(block_sizes))

        # Loop over all combinations of indices
        for inds in it.product(*(range(n) for n in small_shape)):
            #print("inds = {}".format(inds))
            startinds = [ind * block_size for ind, block_size in zip(inds, block_sizes)]
            stopinds = [(ind+1) * block_size for ind, block_size in zip(inds, block_sizes)]
            slices = tuple(slice(startind, stopind) for startind, stopind in zip(startinds, stopinds))
            #print("startinds = {}".format(startinds))
            #print("stopinds = {}".format(stopinds))
            avg[inds] = large_arr[slices].mean()

        return avg
    
    else:
        raise IndexError("`small_shape` must divide `large_arr.shape` elementwise.")

In [ ]:
def grid_study(max_ns, max_na, a_water, b, const, fname, kelp_profile='top-heavy', min_ns=4, min_na=4):
    # Determine which grid sizes to run.
    # Must be a factor of maximum,
    # and na must be even.
    ns_list = [n for n in divisors(max_ns) if n >= min_ns]
    na_list = [n for n in divisors(max_na) if n >= min_na and n%2 == 0]
    
    # Create parameter span
    param_span_name = 'kelp-grid-study'
    psw = ParamSpanWidget(param_span_name, remote_config, kelp_calculate, kelp_visualize)
    
    # Set parameters
    psw.set_params(
        ns = ns_list,
        na = na_list,
        a_water = [a_water],
        b = [b],
        kelp_profile = [kelp_profile],
        const=[const]
    )
    
    # Run computations
    psw.submit_computations()
    
    # Wait for computations to complete
    for fut in psw.save_futures:
        # fut is initially None
        while True:
            if fut:
                # Wait for result and throw away (should be None)
                _ = fut.result()
                break
            
    # Allocate error arrays
    abs_err = np.zeros([len(ns_list), len(na_list)])
    rel_err = np.zeros([len(ns_list), len(na_list)])
    
    #grid_sizes = np.empty([len(ns_list), len(na_list)], dtype=object)
    #ns_grid, na_grid = np.meshgrid(ns_list, na_list, indexing='ij')
    
    # Extract "true" (highest accuracy) radiance
    # Assume that highest index is highest solution
    # (jobs were submitted in order of increasing grid size)
    true_ind = len(ns_list) * len(na_list) - 1
    true_irrad = psw.results[true_ind]['irradiance']
    
    # Compare values
    for i, ns in enumerate(ns_list):
        for j, na in enumerate(na_list):
            # Find the index of this combination
            # Assuming only one row matches both ns and na.
            ind = psw.param_table.df.query('ns == {} and na == {}'.format(ns, na)).index[0]
            
            # Read radiance
            irrad = psw.results[ind]['irradiance']
            
            #print("i, ns = {}".format((i, ns)))
            #print("j, na = {}".format((j, na)))
            #print("irrad shape = {}".format(irrad.shape))
            #print("true_irrad shape = {}".format(true_irrad.shape))
            
            # Block averages of true radiance
            avg_true = block_mean(true_irrad, irrad.shape)
            
            # Calculate error
            abs_err[i, j] = np.mean(np.abs(irrad-avg_true))
            rel_err[i, j] = np.mean(np.abs(irrad-avg_true)/np.abs(avg_true))
            
    
    final_dict = {
        'a_water': a_water,
        'b': b,
        'ns_list': ns_list,
        'na_list': na_list,
        'kelp_profile': kelp_profile,
        'const': const,
        'abs_err': abs_err,
        'rel_err': rel_err,
        'results': psw.results
    }
    
    print("Grid study finished, saving.")
    with open(fname, 'wb') as fh:
        dill.dump(final_dict, fh)
    print("Saved!")
    
    return psw, ns_list, na_list, abs_err, rel_err


In [ ]:
executor = cf.ThreadPoolExecutor(max_workers=4)

In [ ]:
study_results = {}
a_water_list = [0.05, 0.5]
b_list = [0.05, 0.5]
# Submit a few IOP cases
# Don't want to over load memory
for a_water, b in it.product(a_water_list, b_list):
    fname = 'grid88a{:.2f}_b{:.2f}'.format(a_water,b)
    study_results[(a_water, b)] = executor.submit(grid_study, a_water=a_water, fname=fname, b=b, max_ns=24, max_na=24, const=const)

In [ ]:
t = study_results[(0.5,0.5)].result()

In [ ]:
t[0]

In [ ]:
lv = c.load_balanced_view()

In [ ]:
%px !free

In [ ]:
def imp():
    import pandas._libs.sparse as splib
 

In [ ]:
lv.apply(imp).result()

In [ ]:
!pip show numpy ipyparallel pandas python

In [ ]:
study1.result()

In [ ]:
study1.

In [ ]:
[type(r) for r in psw.results]

In [ ]:
gs, ns_list, na_list, abs_err, rel_err = grid_study(max_ns=24, max_na=24, const=const)
gs

In [ ]:
ns_list

In [ ]:
na_list

In [ ]:
abs_err

In [ ]:
rel_err

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ax = plt.gca()
plt.contour(rel_err.T, colors='k', linewidths=5)
plt.contour(rel_err.T, colors='w', linewidths=2)
plt.imshow(rel_err.T)
ax.set_xticks(range(len(ns_list)))
ax.set_yticks(range(len(na_list)))
ax.set_xticklabels(ns_list)
ax.set_yticklabels(na_list)
plt.xlabel('Spatial Grid Size')
plt.ylabel('Angular Grid Size')
plt.title('Relative Error')
plt.colorbar()
plt.tight_layout()